In [2]:
num_sample = 300
num_burn = 20
sample_size = 800

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
min_degree = 1
max_degree = 2
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y2, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

100%|██████████| 320/320 [00:06<00:00, 51.09it/s]


(-0.08226961103096737, 0.2010625)

In [3]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=500,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

100%|██████████| 600/600 [00:08<00:00, 71.33it/s]


{'average': 0.25639375000000003,
 'direct_effect': -0.13969625,
 'spillover_effect': -0.167985,
 'psi_1_gamma': 0.21558124999999997,
 'psi_0_gamma': 0.35527749999999997,
 'psi_zero': 0.5232625}

In [4]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=500,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y2
)

100%|██████████| 600/600 [00:07<00:00, 80.59it/s]


{'average': 0.25677625,
 'direct_effect': -0.0037349999999999883,
 'spillover_effect': -0.16502624999999999,
 'psi_1_gamma': 0.25577500000000003,
 'psi_0_gamma': 0.25951,
 'psi_zero': 0.42453625}

In [5]:
n_cpu = 100

In [6]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

Multiprocessing <function run_autognet at 0x7fdbd2a068e0> in 100 tasks, with 100 processes...
Multiprocessing finished.
average: 0.22573 ± 0.02904
direct_effect: -0.06616 ± 0.02219
spillover_effect: -0.14724 ± 0.03412
psi_0_gamma: 0.26911 ± 0.02630
psi_zero: 0.41635 ± 0.01648
psi_1_gamma: 0.20294 ± 0.03165


In [7]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr at 0x7fdbe9c98360> in 100 tasks, with 100 processes...
Multiprocessing finished.
average: 0.24932 ± 0.05994
direct_effect: -0.02265 ± 0.09446
spillover_effect: -0.15891 ± 0.05688
psi_0_gamma: 0.26659 ± 0.04979
psi_zero: 0.42550 ± 0.02237
psi_1_gamma: 0.24394 ± 0.08398


In [1]:
# from run_pll import *
# from utils import run_pll

# args = [{'Y_chain': Y_chain,
#          'A_chain': A_chain,
#          'L_chain': L_chain,
#          'adj': adj,
#          'i': i} for i in range(len(Y_chain))]

# res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
# res_list_array_dr = np.array(res_list_dr)

# from run_pll import column_names

# ret_mean_dr = res_list_array_dr.mean(axis=0)
# ret_std_dr = res_list_array_dr.std(axis=0)
# for i in range(len(column_names)):
#     print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [1]:
n_cpu = 100

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

sample_size = 1600
num_sample = 300
num_burn = 100

# 1. Simulate or load data
min_degree = 1
max_degree = 2
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y2, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

100%|██████████| 400/400 [00:07<00:00, 52.72it/s]


(-0.10535724757074673, 0.18219375)

In [ ]:


res_truth1 = agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0,
    sample_Y_func=sample_Y2
)
res_truth2 = agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.3,
    R=200,
    burnin_R=50,
    seed=0,
    sample_Y_func=sample_Y2
)

ground_truth1 = {'average': 0, 'direct': 0, 'indirect': 0, 'spillover_effect':0}
ground_truth1['average'] = res_truth1['average']
ground_truth1['direct'] = res_truth1['direct_effect']
ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
ground_truth1['spillover_effect'] = res_truth1['spillover_effect']

ground_truth1


 68%|██████▊   | 171/250 [00:02<00:00, 81.78it/s]

In [ ]:
from utils import run_pll
from run_pll import run_dr_raw

treat_p = 0.7
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': False} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

In [ ]:
from utils import run_pll
from run_pll import run_dr_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': True} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

In [ ]:
# read results
import numpy as np
from utils import compute_stats

results1 = np.load('run/sim_results/sim_dr_raw_800_2_5_0.7.npy', allow_pickle=True)
results2 = np.load('run/sim_results/sim_dr_raw_800_2_5_0.3.npy', allow_pickle=True)

print(compute_stats(results1, results2, ground_truth1, adj))